In [1]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

from matplotlib.colors import LinearSegmentedColormap


import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


In [2]:

latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir
latestdir

# dfAge = pd.read_csv(latestdir+'\\Regionalt_DB\\18_fnkt_alder_uge_testede_positive_nyindlagte.csv',delimiter=';',encoding='latin1')

dfCurAdmitted = pd.read_csv(latestdir+'\\Gennembruds_DB\\01_indlagte_pr_vaccinationsstatus.csv',delimiter=';',encoding='latin1')
df = pd.read_csv(latestdir+'\\Gennembruds_DB\\06_nyindlagte_pr_vaccinationsstatus_pr_aldersgrp_pr_uge.csv',delimiter=';',encoding='latin1')

df['Nyindlagte pr. 100.000'] = pd.to_numeric(df['Nyindlagte pr. 100.000'].str.replace(',','.'))

In [3]:

firstWeek = 20
firstIndexToUse = df[df.Uge == '2021-W20'].index[0]
dfShort = df.loc[firstIndexToUse:]

dfVacc = dfShort[dfShort.Vaccinationsstatus == 'Forventet fuld effekt']
dfUnvacc = dfShort[dfShort.Vaccinationsstatus == 'Ikke vaccineret']

lastWeek = int(df.iloc[-1]['Uge'][-2:])

totNumWeeks = int(lastWeek-firstWeek)
weekRange = np.arange(firstWeek,lastWeek+1)

weekLabels = ['Uge\n'+str(x) for x in weekRange]
weekLabels = ['Uge '+str(x) for x in weekRange]

allAgeGroups = df.Aldersgruppe.unique()

fullMatVaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatVaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfVacc[(dfVacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            
    fullMatVaccInci[:,ageIndex] = ageInci
    fullMatVaccCount[:,ageIndex] = ageCount
# pd.DataFrame(fullMatVaccInci,columns=allAgeGroups)

dfPlotVacc = pd.DataFrame(fullMatVaccInci,columns=allAgeGroups) 



fullMatUnvaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatUnvaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfUnvacc[(dfUnvacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            
    fullMatUnvaccInci[:,ageIndex] = ageInci
    fullMatUnvaccCount[:,ageIndex] = ageCount


dfPlotUnva = pd.DataFrame(fullMatUnvaccInci,columns=allAgeGroups)

In [4]:


curAge = '80+'

# agedf = dfShort[dfShort.Aldersgruppe == curAge]

# Meanwidth
wi = 5

clrVacc = '#004488'
clrUnva = '#884400'


for curAge in allAgeGroups:
    fig,ax1 = plt.subplots()

    ax1.plot(weekRange,dfPlotUnva[curAge],'.:',linewidth=1,color=clrUnva,label='Uvaccinerede, data')
    ax1.plot(rnTime(weekRange,wi),rnMean(dfPlotUnva[curAge],wi),color=clrUnva,label=f'Uvaccinerede, {wi} ugers gennemsnit')
    ax1.plot(weekRange,dfPlotVacc[curAge],'.:',linewidth=1,color=clrVacc,label='Vaccinerede, data')
    ax1.plot(rnTime(weekRange,wi),rnMean(dfPlotVacc[curAge],wi),color=clrVacc,label=f'Vaccinerede, {wi} ugers gennemsnit')

    ax1.legend(loc='upper left')

    ax1.set_ylim(bottom=0)
    ax1.set_xticks(weekRange)
    ax1.set_xticklabels(weekLabels,rotation=90)

    ax1.set_ylabel('Antal nyindlæggelser\nper 100.000 borgere i befolkningsgruppen')
    ax1.set_title('Aldersgruppe: '+curAge)

    # ax1.set_xlim([weekLabels[int(wi/2)],weekLabels[-1]])
    ax1.set_xlim([weekRange[int(wi/2)]+5,weekRange[-1]+0.5])
    # ax1.set_xlim(left=weekRange[int(wi/2)+5])

    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)

    ax1.grid(axis='y')

    SourceText = 'Rasmus K. Pedersen, PandemiX center, RUC - Datakilde: covid19.ssi.dk/overvagningsdata \n Fra dashboard-fil "Gennembruds_DB\\06_nyindlagte_pr_vaccinationsstatus_pr_aldersgrp_pr_uge.csv"'

    # ax1.text(weekRange[-1]+0.5,-30,SourceText,ha='right',fontsize=7)
    ax1.text(1,-0.2,SourceText,ha='right',fontsize=7,transform = ax1.transAxes)
    fig.tight_layout()

    if saveFigures:
        plt.savefig(path_figs+f'Breakthrough/HospOverTime_{curAge}.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# Effectivity over time

# for curAge in allAgeGroups:
    # fig,ax1 = plt.subplots()

    # ax1.plot(weekRange,dfPlotUnva[curAge],'.:',linewidth=1,color=clrUnva,label='Uvaccinerede, data')
    # ax1.plot(rnTime(weekRange,wi),rnMean(dfPlotUnva[curAge],wi),color=clrUnva,label=f'Uvaccinerede, {wi} ugers gennemsnit')
    # ax1.plot(weekRange,dfPlotVacc[curAge],'.:',linewidth=1,color=clrVacc,label='Vaccinerede, data')
    # ax1.plot(rnTime(weekRange,wi),rnMean(dfPlotVacc[curAge],wi),color=clrVacc,label=f'Vaccinerede, {wi} ugers gennemsnit')

    # ax1.legend(loc='upper left')

    # ax1.set_ylim(bottom=0)
    # ax1.set_xticks(weekRange)
    # ax1.set_xticklabels(weekLabels,rotation=90)

    # ax1.set_ylabel('Antal nyindlæggelser\nper 100.000 borgere i befolkningsgruppen')
    # ax1.set_title('Aldersgruppe: '+curAge)

    # # ax1.set_xlim([weekLabels[int(wi/2)],weekLabels[-1]])
    # ax1.set_xlim([weekRange[int(wi/2)]+5,weekRange[-1]+0.5])
    # # ax1.set_xlim(left=weekRange[int(wi/2)+5])

    # ax1.spines['top'].set_visible(False)
    # ax1.spines['right'].set_visible(False)

    # ax1.grid(axis='y')

    # SourceText = 'Rasmus K. Pedersen, PandemiX center, RUC - Datakilde: covid19.ssi.dk/overvagningsdata \n Fra dashboard-fil "Gennembruds_DB\\06_nyindlagte_pr_vaccinationsstatus_pr_aldersgrp_pr_uge.csv"'

    # # ax1.text(weekRange[-1]+0.5,-30,SourceText,ha='right',fontsize=7)
    # ax1.text(1,-0.2,SourceText,ha='right',fontsize=7,transform = ax1.transAxes)
    # fig.tight_layout()

curAge = '80+'

wi = 5
rnTi = rnTime(weekRange,wi)

fig,ax1 = plt.subplots()


cmap = plt.cm.get_cmap('Dark2',len(allAgeGroups))

counter = 0
for curAge in allAgeGroups:
    rnUn = rnMean(dfPlotUnva[curAge],wi)
    rnVa = rnMean(dfPlotVacc[curAge],wi)
    # ax1.plot(rnTi,1-(rnVa/rnUn),label=curAge)
    ax1.plot(rnTi,100*(1-(rnVa/rnUn)),label=curAge,color=cmap(counter))

    counter = counter + 1 

# ax1.set_ylim(bottom=0,top=1)
ax1.set_ylim(bottom=60,top=101)
ax1.set_xticks(weekRange)
ax1.set_xticklabels(weekLabels,rotation=90)

ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

ax1.grid(axis='y')

ax1.set_ylabel(f'Vaccineeffektivitet i forhold til indlæggelse [%]\n {wi} ugers rullende gennemsnit')
ax1.legend(loc='lower left')


SourceText = 'Rasmus K. Pedersen, PandemiX center, RUC - Datakilde: covid19.ssi.dk/overvagningsdata \n Fra dashboard-fil "Gennembruds_DB\\06_nyindlagte_pr_vaccinationsstatus_pr_aldersgrp_pr_uge.csv"'
ax1.text(1,-0.2,SourceText,ha='right',fontsize=7,transform = ax1.transAxes)

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …